In [ ]:
from core import Wrap_PyBamm
%load_ext autoreload
%autoreload 2

In [ ]:
path = "here.h5"
datapath = "/home/ccanodom/data/EV_battery/battery.h5"

In [ ]:
res = Wrap_PyBamm.run_iterative(20, datapath, path,12)

In [ ]:
res = Wrap_PyBamm.run_iterative(20, datapath, path,12)

In [ ]:
res

In [ ]:
(df_experimenta, dict_df_PB) = Wrap_PyBamm.run_iterative(20, datapath, path,3)

### dict_df_PB.pop(

In [ ]:
get_metrics(BatteryIDEV.W10, datapath,cycle_number= 5, solution=solution)

In [ ]:
relative_time = solution['Time [h]'].entries - np.min(solution['Time [h]'].entries )
C =  -1 * solution['Current [A]'].entries
V = solution['Battery voltage [V]'].entries
data = solution.get_data_dict(None)
Cycle = data["Cycle"] 
Step = data['Step'][-len(Cycle):]

In [ ]:
cycle_dataframes = {}
cycles = np.unique(Cycle)
cycles = cycles[cycles != 0.0]
# Loop over each unique cycle number
for cycle_number in cycles:
    # Find the indices for the current cycle number
    indices = np.where(Cycle == cycle_number)[0]
    
    # Extract the data for the current cycle
    cycle_relative_time = relative_time[indices] - np.min(relative_time[indices] )
    cycle_C = C[indices]
    cycle_V = V[indices]
    cycle_Step = Step[indices] 
    
    # Create a dataframe for the current cycle
    cycle_df = pd.DataFrame({
        'relative_time': cycle_relative_time,
        'C': cycle_C,
        'V': cycle_V,
        'Step': cycle_Step
    })
    
    # Add the dataframe to our dictionary
    cycle_dataframes[cycle_number] = cycle_df

In [ ]:
for cycle_index, df_simulated in dict_df_PB.items():
        print(cycle_index)
        print(df_simulated)
        results +=  calc_error(df_experimental, df_simulated, 1, "C")
        results += calc_error(df_experimental, df_simulated, 2, "V")
        results += calc_error(df_experimental, df_simulated, 3, "C")
        results += calc_error(df_experimental, df_simulated, 4, "V")


In [ ]:
solution
import pandas as pd
import numpy as np
relative_time = solution['Time [h]'].entries - np.min(solution['Time [h]'].entries )
C =  -1 * solution['Current [A]'].entries
V = solution['Battery voltage [V]'].entries
data = solution.get_data_dict(None)
Cycle = data["Cycle"] 
Cycle = Cycle[Cycle != 0.0]
Step = data['Step'][-len(Cycle):]

cycle_dataframes = {}

# Loop over each unique cycle number
for cycle_number in np.unique(Cycle):
    # Find the indices for the current cycle number
    indices = np.where(Cycle == cycle_number)[0]
    
    # Extract the data for the current cycle
    cycle_relative_time = relative_time[indices]
    cycle_C = C[indices]
    cycle_V = V[indices]
    cycle_Step = Step[indices] if len(Step) == len(Cycle) else [None] * len(indices)  # Adjust based on Step data availability
    
    # Create a dataframe for the current cycle
    cycle_df = pd.DataFrame({
        'relative_time': cycle_relative_time,
        'C': cycle_C,
        'V': cycle_V,
        'Step': cycle_Step
    })
    
    # Add the dataframe to our dictionary
    cycle_dataframes[cycle_number] = cycle_df


In [ ]:
df_experimental = get_df_experimental(BatteryIDEV.W10, datapath, 5)

In [ ]:
from databases.EVExperiment import *
dict_df_PB = get_df_simulated(solution)
results = 0.0 #(df_experimental, dict_df_PB)
for cycle_index, df_simulated in dict_df_PB.items():
    print(cycle_index)
    results +=  calc_error(df_experimental, df_simulated, 1, "C")
    results += calc_error(df_experimental, df_simulated, 2, "V")
    results += calc_error(df_experimental, df_simulated, 3, "C")
    results += calc_error(df_experimental, df_simulated, 4, "V")
results

In [ ]:
from databases import EVExperiment
from databases.EVExperiment import * 
df_simulated = cycle_dataframes[1.0]
df_experimental = EVExperiment.get_df_experimental(EVExperiment.BatteryIDEV.W10, datapath, 4)
battery_id = EVExperiment.BatteryIDEV.W10
path = datapath
cycle_number = 5


In [ ]:
df_experimental = get_df_experimental(battery_id, path, cycle_number)
results = 0.0
dict_df_PB = get_df_simulated(solution)
for cycle_index, df_simulated in dict_df_PB.items():
    print(cycle_index)
    results +=  calc_error(df_experimental, df_simulated, 1, "C")
    results += calc_error(df_experimental, df_simulated, 2, "V")
    results += calc_error(df_experimental, df_simulated, 3, "C")
    results += calc_error(df_experimental, df_simulated, 4, "V")

 

In [ ]:
parameter_values = pybamm.ParameterValues("OKane2022")
parameter_values

In [ ]:
string_id = "decay"
cycle = 10.0

In [ ]:
f"/{string_id}_Cycle{int(cycle):04d}"

In [ ]:
import h5py
import pandas as pd

In [ ]:
datapath = "/home/ccanodom/data/EV_battery/battery.h5"
from databases import EVExperiment
battery_id = EVExperiment.BatteryIDEV.W04
EVExperiment.read_dataframe(battery_id, datapath, 10)
pd.read_hdf("here.h5", )

In [ ]:
with h5py.File("here.h5", 'r') as file:
    for i in file.keys():
        print(i)
df = pd.read_hdf("here.h5", "decay_exp_3_Cycle0001")

In [ ]:
df = pd.read_hdf("here.h5", key = "decay_exp_3_Cycle0001")

In [ ]:
df["relative_time"] = df["Time"] - df["Time"].min()

In [ ]:
# Leer y combinar los subdatasets especificados en un único dataframe
with h5py.File(datapath, 'r') as file:
    relative_time = file['W04_Cycle0010']['relative_time'][:].squeeze()
    Step = file['W04_Cycle0010']['Step'][:].squeeze()
    C = file['W04_Cycle0010']['C'][:].squeeze()
    V = file['W04_Cycle0010']['V'][:].squeeze()

In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

def calc_error(EV_data, PB_data, step, var, to_plot=False, interp_points=100):
    """
    Calcula el error RMSE entre los datos de un vehículo eléctrico y una simulación PyBamm.

    :param EV_data: DataFrame de datos del vehículo eléctrico.
    :param PB_data: DataFrame de datos de la simulación PyBamm.
    :param step: Paso específico para el análisis.
    :param cycle: Ciclo específico para el análisis.
    :param var: Variable a analizar ('C' o 'V').
    :param to_plot: Si es True, traza las curvas de EV y PB.
    :param interp_points: Número de puntos para la interpolación.
    :return: RMSE normalizado por el rango de la variable.
    """
    # Filtrar datos por ciclo y paso
    compEV = EV_data[(EV_data['Step'] == step)]
    compEV.loc[:, 'relative_time'] = compEV['relative_time'] - compEV['relative_time'].min()

    
    compPB = PB_data[(PB_data['Step'] == step)]
    compPB.loc[:, 'relative_time'] = compPB['relative_time'] - compPB['relative_time'].min()

    # Rango para normalizar la importancia de cada segmento
    range_min = min(compPB[var].min(), compEV[var].min())
    range_max = max(compPB[var].max(), compEV[var].max())
    value_range = range_max - range_min

    # Tiempo común entre las dos curvas
    min_time = max(compPB['relative_time'].min(), compEV['relative_time'].min())
    max_time = min(compPB['relative_time'].max(), compEV['relative_time'].max())
    trunc_time = np.linspace(min_time, max_time, interp_points)

    # Interpolación en el vector de tiempo común
    interp_EV = interp1d(compEV['relative_time'], compEV[var], fill_value="extrapolate")
    interp_PB = interp1d(compPB['relative_time'], compPB[var], fill_value="extrapolate")
    EV_y = interp_EV(trunc_time)
    PB_y = interp_PB(trunc_time)

    # Calcular el error RMSE
    calc_rmse = np.sqrt(np.mean((PB_y - EV_y) ** 2)) / value_range

    # Opcional: trazar las curvas
    if to_plot:
        plt.figure(figsize=(10, 6))
        plt.plot(trunc_time, EV_y, label='EV Data')
        plt.plot(trunc_time, PB_y, label='PB Data')
        plt.legend()
        plt.xlabel('Relative Time')
        plt.ylabel(var)
        plt.title(f'Comparison of {var} over Time')
        plt.show()

    return calc_rmse

In [ ]:
calc_error(df_combined, df,2,"V", True,10)

In [ ]:
# Crear el dataframe combinado
df_combined = pd.DataFrame({
    'relative_time': relative_time,
    'Step': Step,
    'C': C,
    'V': V
})

In [ ]:
df_combined

In [ ]:
def read_cycle_from_hdf5(cycle_number, string_id, filepath):
    """
    Reads and returns the DataFrame of a specific cycle from an HDF5 file.
    
    Parameters:
    - cycle_number: Integer representing the cycle number to read.
    - string_id: String ID used as part of the key to identify the cycle data.
    - filepath: Path to the HDF5 file from which to read the data.
    
    Returns:
    - A pandas DataFrame containing the data for the specified cycle, or None if the cycle data does not exist.
    """
    key = f'/{string_id}_Cycle{int(cycle_number):04d}'
    print(key)
    try:
        cycle_data = pd.read_hdf(filepath, key=key)
        return cycle_data
    except KeyError:
        print(f"No data found for cycle {cycle_number} with ID '{string_id}' in the file.")
        return None
